In [99]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from datetime import timedelta
pd.set_option('display.max_rows',100)
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.width',1000)
pd.set_option('display.max_columns',20)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


## Cálculo do vol acumulado médio - periodo 60d, interval 5m

In [ ]:
pd.set_option('display.max_rows',50)

def calc_volume(tick,period,interval):
    m5 = yf.Ticker(tick+".SA").history(period=period,interval=interval, prepost = True).reset_index()
    m5['ticker']=tick
    #trata horário de verão
    m5.loc[m5.Datetime.dt.date < pd.to_datetime("2021-02-22").date() , 'Datetime']  = (m5.Datetime - timedelta(minutes=60))
    m5.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    m5['Volume']=m5.Volume/100#1000000
    m5['vol_acumulado'] = m5.groupby(m5.Datetime.dt.date)['Volume'].cumsum()
    m5['hour']=m5.Datetime.dt.hour
    m5['minute']=m5.Datetime.dt.minute


    m5_media_qtd = pd.merge(
        m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].mean().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'vol_acum_medio'})
        ,m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].count().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'qtd_vol_acum'})
        ,how='inner', on=['hour','minute']
             )

    m5_merge = pd.merge(m5,m5_media_qtd,how='left',on=['hour','minute'])
    m5_merge['pct_vol_acumulado'] = m5_merge.vol_acumulado / m5_merge.vol_acum_medio*100
    m5_merge=m5_merge.round(2)
    
    return m5_merge

In [ ]:
ibr=pd.read_csv('./ibr.txt')#.tail(2)

period="60d"
interval="5m"

for i, row in ibr.iterrows(): 
    if i == 0:
        df = calc_volume(row.ticker,period,interval)
    else:
        df = df.append(calc_volume(row.ticker,period,interval))

In [ ]:
df[
    (       df.Datetime.dt.date == pd.to_datetime("today").date()  )
        &
    (  df.pct_vol_acumulado > 200      )
]

## Diário

In [173]:
def diario(tick, period):
    acao=tick+".SA"
    df = yf.Ticker(acao).history(period=period)
    df['Acao'] = tick
    df['cotacao_hj'] = yf.Ticker(acao).history(period="1m").tail(1).Close.iloc[0]
    df['cotacao_dia_anterior'] = df['Close'].shift(1)
    df['pct_do_dia'] = (df.Close - df.cotacao_dia_anterior) / df.cotacao_dia_anterior*100
    df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    df.Volume = df.Volume/1000000
    df['media_vol'] = df.Volume.mean()
    df['var_vol'] = df.Volume / df.media_vol
    df['pct_longo'] = (df.cotacao_hj-df.Low)/df.cotacao_hj*100
    df.loc[df['pct_do_dia'] < 0, 'pct_longo'] = (df.High-df.cotacao_hj)/df.cotacao_hj*100

    return df

In [174]:
%%time
period="120d"
ibr=pd.read_csv('./ibr.txt')#.head(2)
for i, row in ibr.iterrows():
    if i == 0:
        df = diario(row.ticker,period)
    else:
        df = df.append(diario(row.ticker,period))

df=df.reset_index()
df=df[['Acao','Date','Open','High','Low','Close','cotacao_hj','cotacao_dia_anterior','pct_do_dia','media_vol','Volume','var_vol','pct_longo']]           
df = df.sort_values(by=['Acao','Date'],ascending=False).round(2)

CPU times: user 20.6 s, sys: 363 ms, total: 20.9 s
Wall time: 1min 57s


In [172]:
df[
    (df.pct_do_dia >= 4)
    #(df.pct_do_dia.abs() >= 4)
   & (abs(df.pct_longo) <= 4)
   & (df.var_vol > 2.5)
   & (df.Date.dt.date <= pd.to_datetime('today').date() - timedelta(days=3) )
   #& (df.media_vol>0.5)
  ].sort_values(['Date','var_vol'],ascending=[False , False]).round(2)

,Acao,Date,Open,High,Low,Close,cotacao_hj,cotacao_dia_anterior,pct_do_dia,media_vol,Volume,var_vol,pct_longo
5095,LAME3,2021-02-22,24.21,26.74,23.21,26.39,24.01,18.85,40.00,2.06,19.36,9.41,3.33
5154,LAME4,2021-02-22,27.10,29.26,25.83,28.95,25.98,24.15,19.88,10.98,64.14,5.84,0.58
5036,KLBN11,2021-02-22,29.47,30.98,28.88,30.90,29.78,29.23,5.71,5.09,23.57,4.63,3.02
5743,LWSA3,2021-02-19,31.29,35.06,30.93,34.99,29.81,30.53,14.61,7.43,21.34,2.87,-3.76
3905,GFSA3,2021-02-03,4.81,5.62,4.79,5.42,4.80,4.82,12.45,10.09,47.92,4.75,0.21
4137,GOLL4,2021-01-28,23.30,25.76,23.30,24.61,22.51,23.03,6.86,8.74,24.59,2.81,-3.51
4007,GNDI3,2021-01-11,90.00,103.04,88.80,101.45,88.29,91.40,11.00,5.07,18.28,3.61,-0.58
621,AZUL4,2020-12-03,41.14,44.18,41.12,42.28,42.45,40.52,4.34,8.87,23.02,2.59,3.13
5517,LOGN3,2020-12-02,16.28,17.50,16.26,17.26,15.66,16.21,6.48,1.02,4.14,4.07,-3.83
2861,CYRE3,2020-12-01,26.36,27.61,25.86,27.37,25.15,25.73,6.39,6.10,15.76,2.58,-2.83


## Gráfico

In [ ]:
brf = yf.Ticker("BRFS3.SA").history(period='1y')
brf = brf.reset_index()

for i in ['Open','High','Close','Low']:
    brf[i] = brf[i].astype('float64')
    
fig = go.Figure(data=[go.Candlestick(x=brf['Date'],open=brf['Open'],high=brf['High'],low=brf['Low'],close=brf['Close'])])
#fig.show()

## Get .info

In [ ]:
ibr=pd.read_csv('./ibr.txt').tail(2)

df = pd.DataFrame(columns = ['Tick', 'profitMargins'])

for i, row in ibr.iterrows(): 
    acao = row.ticker+".SA"
    df.loc[i] = row.ticker, yf.Ticker(acao).info['profitMargins']